In [1]:
from openai import OpenAI

client = OpenAI()

import argparse
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return client.chat.completions.create(**kwargs)

In [2]:
import wandb
from types import SimpleNamespace
from tqdm.auto import tqdm
import pandas as pd

WANDB_PROJECT = "alpaca-ft"
WANDB_ENTITY = "kevinv3796"
INPUT_DATASET_TABLE_AT = dict(
    table_at = 'kevinv3796/alpaca-ft/run-8paqy77d-eval_predictions:latest',
    table_name = "eval_predictions"
)

config = SimpleNamespace(
    max_tokens=256,
    system_prompt="You are a helpful assistant.",
    temperature=1,
    prompt_col="prompt",
    input_dataset_table_at=INPUT_DATASET_TABLE_AT,
    output_table="gpt35_results",
    num_samples=250,  # for debug purposes
)

def generate_35(prompt, config):
    completion = completion_with_backoff(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": config.system_prompt},
                  {"role": "user","content": prompt}
                 ],
        max_tokens=config.max_tokens,
        temperature=config.temperature,

    )
    output = completion.choices[0].message.content
    # print(f"##PROMPT: {prompt}\n##COMPLETION: {output}")
    return output

def download_table(table_at, table_name):
    artifact = wandb.use_artifact(table_at, type='run_table')
    artifact_dir = artifact.download()
    table = artifact.get(table_name)
    df = pd.DataFrame(data=table.data, columns=table.columns)
    return df


# create a run to have lineage
wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, job_type="eval", tags=["gpt-3.5"], config=config)

input_df = download_table(**config.input_dataset_table_at)
results = []
for i, prompt in enumerate(tqdm(input_df[config.prompt_col].to_list()[:config.num_samples])):
    results.append((i, generate_35(prompt, config)))
gpt35_df = pd.DataFrame(results, columns=["index", "generation"])

# format to save
gpt35_df = gpt35_df.assign(prompt=input_df[config.prompt_col])
gpt35_df = gpt35_df[["index", "prompt", "generation"]]
print(gpt35_df.head())

gpt35_table = wandb.Table(dataframe=gpt35_df)

# save our work's output!
wandb.log({config.output_table:gpt35_table})

wandb: Currently logged in as: kevinv3796. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


  0%|          | 0/250 [00:00<?, ?it/s]

   index                                             prompt  \
0      0  Below is an instruction that describes a task....   
1      1  Below is an instruction that describes a task....   
2      2  Below is an instruction that describes a task....   
3      3  Below is an instruction that describes a task....   
4      4  Below is an instruction that describes a task....   

                                          generation  
0  Certainly! Here are some creative ways you can...  
1  A suggestion for a research paper topic relate...  
2                          The sum of 9 and 4 is 13.  
3  Here are four countries in Africa:\n\n1. Niger...  
4  Sure! Here are five examples of healthy snacks...  


In [3]:
import wandb
import json
from pathlib import Path
from types import SimpleNamespace
from tqdm.auto import tqdm
import pandas as pd

WANDB_PROJECT = "alpaca-ft"
WANDB_ENTITY = "kevinv3796"

FT_MODEL_PREDS_AT = dict(
    table_at = 'kevinv3796/alpaca-ft/run-8paqy77d-eval_predictions:latest',
    table_name="eval_predictions",
)
GPT35_MODEL_PREDS_AT = dict(
    table_at='kevinv3796/alpaca-ft/run-dhovpwxl-gpt35_results:latest',
    table_name='gpt35_results',
)


config = SimpleNamespace(
    openai_model = "gpt-4",
    temperature=1,
    system_prompt = ("You will be presented with a choice of two possible responses for an instruction"
                     "You have to pick the best one and give a reason why.\n"
                     "The reponse should follow the instructions and use the provided context if there is some\n"
                     "If both answers are equivalent, pick the value 0"),
    model_names=["ft_model", "gpt35"],
    num_samples=250,
    out_dir="./output",
)

def download_table(table_at, table_name):
    artifact = wandb.use_artifact(table_at, type='run_table')
    artifact_dir = artifact.download()
    table = artifact.get(table_name)
    df = pd.DataFrame(data=table.data, columns=table.columns)
    return df

def gpt4_judge(instruction, gen1, gen2, model=config.openai_model, system_prompt=config.system_prompt):
    message = "{instruction}\n Answer 1: \n{gen1}\n Answer 2:\n{gen2}".format(instruction=instruction, gen1=gen1, gen2=gen2)
    completion = completion_with_backoff(
        model="gpt-4",
        messages=[{"role": "system",
                   "content": system_prompt,
                  },
                  {"role": "user",
                   "content": message,
                  },],
        function_call = {"name": "make_choice"},
        functions = [
            {
                "name": "make_choice",
                "description": "Select the best generation and argument why",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "choice": {
                            "type": "integer",
                            "description": "the choosen alternative, zero if equivalent",
                        },
                        "reason":{
                            "type": "string",
                            "description": "Reason why the choice was made",
                        },
                    }
                },
                    "required": ["choice", "reason"],
            },
        ],
    )
    return completion

def judge_row(row, model1_name, model2_name):
    "Judge with inversion of prompt order (2x more expensive)"
    prompt = row.prompt
    gen1 = row[model1_name]
    gen2 = row[model2_name]
    res = gpt4_judge(prompt, gen1, gen2)
    res_inverted = gpt4_judge(prompt, gen2, gen1)
    return res, res_inverted

def extract_function_call(res):
    "Extract the function call arguments"
    try:
        response = json.loads(res.choices[0].message.function_call.arguments)
        choice = response["choice"]
        reason = response["reason"]
    except:
        choice = -1
        reason = "gpt4 fail"
    return choice, reason

def judge(merged_df, model1_name, model2_name):
    "Apply the judge stuff!"
    gpt4_results = []
    for i in tqdm(range(len(merged_df))):
        row = merged_df.iloc[i]
        res, res_inverted = judge_row(row, model1_name, model2_name)
        choice, reason = extract_function_call(res)
        print(choice, reason)
        choice_inverted, reason_inverted = extract_function_call(res_inverted)
        print(choice_inverted, reason_inverted)
        agree = (choice - choice_inverted) % 2 == 1
        # print(f"GPT4 prefers: {choice} and {choice_inverted}: Agree={agree}")
        gpt4_results.append((i, row.prompt, row[model1_name], row[model2_name], choice, choice_inverted, reason, reason_inverted))
    return gpt4_results

def agree_check(row):
    "Check where the GPT4 agrees with itself"
    if row.choice == 0 and row.choice_inverted==0:
        return True
    elif ((row.choice - row.choice_inverted) % 2 == 1):
        return True
    else:
        return False

In [4]:
out_dir = Path(config.out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
# create a run to have lineage
wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, job_type="eval", tags=["gpt-4"], config=config)

gpt35_df = download_table(**GPT35_MODEL_PREDS_AT)
ft_results_df = download_table(**FT_MODEL_PREDS_AT)

# merge both
merged_df = pd.merge(
    ft_results_df[["prompt", "generation"]],
    gpt35_df[["prompt", "generation"]], on="prompt", suffixes=config.model_names,
)
model1_name, model2_name = (f"generation{s}" for s in config.model_names)
gpt_results = judge(merged_df.iloc[:config.num_samples], model1_name, model2_name)
results_df = pd.DataFrame(
    gpt_results,
    columns=["index", "prompt", config.model_names[0], config.model_names[1],
              "choice", "choice_inverted", "reason", "reason_inverted"]).set_index("index")
results_df["agree"] = results_df.apply(agree_check, axis=1)

final_results = results_df[results_df.agree]
print(f"The judge agrees on: {len(final_results)} / {len(ft_results_df)}")

# Let's log those also:
disagree_df = results_df[~results_df.agree]

# Lets use a better naming than 0,1,2
choices = ["both",] + config.model_names
final_results["choice_name"] = [choices[c] for c in final_results["choice"]]
print("\n### GPT JUDGE RESULTS ###")
print("###########################")
print(final_results["choice_name"].value_counts())
print("###########################")

final_results.to_csv(out_dir/"gpt4_eval.csv")
gpt4_table = wandb.Table(dataframe=final_results)
wandb.log({"gpt4_eval":gpt4_table})

disagree_df["choice"] = [choices[c] for c in disagree_df["choice"]]
choices_inverted = ["both",] + config.model_names[::-1]
disagree_df["choice_inverted"] = [choices_inverted[c] for c in disagree_df["choice_inverted"]]

disagree_df.to_csv(out_dir/"gpt4_eval_disagree.csv")
gpt4_eval_disagree_table  = wandb.Table(dataframe=disagree_df)
wandb.log({"gpt4_eval_disagree":gpt4_eval_disagree_table})


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


  0%|          | 0/250 [00:00<?, ?it/s]

2 Answer 2 provides a comprehensive list of creative ways to learn a language, including film and TV, apps, language exchange, music and podcasts, literature, meetups, games, and cooking with foreign-language recipes. Answer 1 simply repeats the same information about Duolingo four times, which does not contribute to the diversity of methods requested in the instruction.
1 Answer 1 is far more detailed and provides many different methods of learning a language, providing learners with more options. It is also clear and well-structured. Answer 2 repeats the same information about Duolingo four times, failing to provide more than one method and being repetitive and redundant.
0 Both responses are equivalent. Each provides a detailed, focused, and relevant research topic for a paper on international relations. They give comprehensive descriptions of the topics with specific points that can be explored regarding globalization and state sovereignty.
2 Answer 2 is more thorough in explaining

<ipython-input-4-b3b5ae605f7c>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_results["choice_name"] = [choices[c] for c in final_results["choice"]]
<ipython-input-4-b3b5ae605f7c>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  disagree_df["choice"] = [choices[c] for c in disagree_df["choice"]]
<ipython-input-4-b3b5ae605f7c>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [6]:
wandb.finish()